In [ ]:
def shuffle_minibatch(x, y):
    assert x.size(0)== y.size(0)
    indices = torch.randperm(x.size(0))
    return x[indices], y[indices]

CUTMIX_ALPHA = 1

In [ ]:
w = 224, h = 224
epoch = 200

In [ ]:
%%time

epochs = 200

for idx,(train_index, valid_index) in enumerate(kfold.split(df_train['img_file'],df_train['class'])):
    print("fold", idx)
    
    x_train_ = df_train.iloc[train_index,:]
    x_valid_ = df_train.iloc[valid_index,:]    
    y_true = x_valid_['class'].values 
    
    train_dataset =  KaggleDataset(x_train_, mode = 'train', path = TRAIN_CROPPED_PATH,
                              transforms = data_transforms['train']
                              )
    valid_dataset =  KaggleDataset(x_valid_, mode = 'valid', path = TRAIN_CROPPED_PATH,
                                  transforms = data_transforms['valid']
                                  )
    # data loader
    train_loader = DataLoader(train_dataset, batch_size = BATCH , shuffle =True)#, num_workers =3,pin_memory=True )
    valid_loader = DataLoader(valid_dataset, batch_size = BATCH , shuffle =False)#, num_workers =3 )
    
    early_stopping = EarlyStopping(patience=15, verbose=True,number=idx)
    model = models.resnet101(pretrained=True)
    model.fc =nn.Linear(2048, num_classes)
    model.to(device)#.cuda()
    learning_rate = 0.001
    criterion = torch.nn.CrossEntropyLoss()#.to(device) # Softmax is internel computed
    #criterion = arcloss(features)
    #criterion = ArcMarginProduct(196, 512, s=30, m=0.5, easy_margin=True)

    optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)
    scheduler = ReduceLROnPlateau(optimizer,'min',factor=0.7, patience=3, min_lr=1e-10,verbose=True)




    for epoch in range(epochs):
        print(epoch)
        model.train() # set the model to train mode(dropout=True)

        train_loss = 0.
        valid_loss = 0.
        valid_preds = np.zeros((len(valid_dataset), num_classes))

        optimizer.zero_grad()
        for i, (inputs, targets) in enumerate(train_loader):

            inputs, targets = inputs.to(device), targets.to(device)                       
            
            
            #-------------------------------------------------------
            # CUTMIX
            #h ttps://www.kaggle.com/kaushal2896/cifar-10-simple-cnn-with-cutmix-using-pytorch
                        
            cutmix_decision = np.random.rand()
            if cutmix_decision > 0.50:
                # Cutmix: https://arxiv.org/pdf/1905.04899.pdf
                x_train_shuffled, y_train_shuffled = shuffle_minibatch(inputs, targets)
                lam = np.random.beta(CUTMIX_ALPHA, CUTMIX_ALPHA)
                cut_rat = np.sqrt(1. - lam)
                
                cut_w = np.int(W * cut_rat)
                cut_h = np.int(H * cut_rat)

                # uniform
                cx = np.random.randint(W)
                cy = np.random.randint(H)


                bbx1 = np.clip(cx - cut_w // 2, 0, W)
                bby1 = np.clip(cy - cut_h // 2, 0, H)
                bbx2 = np.clip(cx + cut_w // 2, 0, W)
                bby2 = np.clip(cy + cut_h // 2, 0, H)

                inputs[:, :, bbx1:bbx2, bby1:bby2] = x_train_shuffled[:, :, bbx1:bbx2, bby1:bby2]
                lam = 1 - (bbx2 - bbx1) * (bby2 - bby1) / (W * H)

            # Forward pass
            y_preds = model(inputs)

            # Calculate loss
            '''
            if cutmix_decision > 0.50:
                loss = criterion(y_preds, targets) * lam + criterion(y_preds, y_train_shuffled) * (1. - lam)
            else:
                loss = criterion(y_preds, targets)
            if i+1 == total_steps:
                loss_list.append(loss)

            '''

            
            
            # CUT MIX
            #----------------------------------------------------------
            
            
            outputs = model(inputs)
            loss = criterion(outputs, targets)


            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            train_loss += loss.item() / len(train_loader)

        # validation
        model.eval() # 평가모드로 전환 dropoput False
        with torch.no_grad():
            for i, (inputs, targets) in enumerate(valid_loader):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                valid_loss += loss.item() / len(valid_loader)
                valid_preds[i * BATCH: (i+1) * BATCH] = outputs.cpu().numpy()
            y_pred = np.argmax(valid_preds, axis=1)
            val_score = f1_score(y_true, y_pred, average='micro')  

        print("Epoch {} , train_loss {:.4f}, valid_loss {:.4f}, val_f1_score {:.4f}".format(epoch, train_loss, valid_loss, val_score))

        scheduler.step(valid_loss)

        early_stopping(valid_loss, model)

        if early_stopping.early_stop:
            print("Early stopping")
            break
    del model
    gc.collect()
     